In [14]:
#write pip import commands
!pip install requests


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import json
import csv
import requests
from urllib.parse import urlencode, quote_plus
import time


In [17]:

# Load profile data
with open("profile_data.json", "r") as file:
    profile = json.load(file)

roles = profile['Desired Roles']
cities = profile['Desired City']

# Define job search URLs for different platforms
job_platforms = {
    "linkedin": "https://www.linkedin.com/jobs/search/",
    "indeed": "https://www.indeed.co.in/jobs",
    "naukri": "https://www.naukri.com/",
    "glassdoor": "https://www.glassdoor.co.in/Job/jobs.htm"
}

# Prepare queries
queries = []
for city in cities:
    for role in roles:
        query = {
            "role": role,
            "city": city
        }
        queries.append(query)

def generate_job_links(role, city):
    """Generate job search links for different platforms"""
    links = {}
    
    # LinkedIn job search URL
    linkedin_params = {
        'keywords': role,
        'location': f"{city}, India",
        'trk': 'public_jobs_jobs-search-bar_search-submit',
        'position': '1',
        'pageNum': '0'
    }
    links['linkedin'] = f"{job_platforms['linkedin']}?{urlencode(linkedin_params)}"
    
    # Indeed job search URL
    indeed_params = {
        'q': role,
        'l': f"{city}, India"
    }
    links['indeed'] = f"{job_platforms['indeed']}?{urlencode(indeed_params)}"
    
    # Naukri job search URL (basic format)
    naukri_query = f"{role.replace(' ', '-')}-jobs-in-{city.lower()}"
    links['naukri'] = f"{job_platforms['naukri']}{naukri_query}"
    
    # Glassdoor job search URL
    glassdoor_params = {
        'sc.keyword': role,
        'locT': 'C',
        'locId': '1',  # India location ID
        'jobType': ''
    }
    links['glassdoor'] = f"{job_platforms['glassdoor']}?{urlencode(glassdoor_params)}"
    
    return links

def search_jobs_with_links(query):
    """Enhanced search function that includes job links"""
    role = query['role']
    city = query['city']
    
    print(f"Searching for {role} jobs in {city}...")
    
    # Generate search links for different platforms
    job_links = generate_job_links(role, city)
    
    # Create job entry with links
    job_entry = {
        "title": f"{role} at Various Companies",
        "location": city,
        "linkedin_link": job_links['linkedin'],
        "indeed_link": job_links['indeed'],
        "naukri_link": job_links['naukri'],
        "glassdoor_link": job_links['glassdoor'],
        "search_query": f"{role} in {city}"
    }
    
    return [job_entry]

def search_jobs_with_api_simulation(query):
    """Simulate API responses with realistic job data including links"""
    role = query['role']
    city = query['city']
    
    print(f"Searching for {role} jobs in {city}...")
    
    # Simulate multiple job results with different companies
    companies = ["TCS", "Infosys", "Wipro", "Accenture", "IBM", "Microsoft", "Google", "Amazon"]
    results = []
    
    for i, company in enumerate(companies[:3]):  # Limit to 3 results per search
        job_id = f"{role.replace(' ', '_').lower()}_{city.lower()}_{company.lower()}_{i}"
        
        # Generate realistic job links
        job_links = generate_job_links(f"{role} {company}", city)
        
        job_entry = {
            "title": f"{role} at {company}",
            "company": company,
            "location": city,
            "job_id": job_id,
            "linkedin_link": job_links['linkedin'],
            "indeed_link": job_links['indeed'],
            "naukri_link": job_links['naukri'],
            "glassdoor_link": job_links['glassdoor'],
            "application_link": f"https://careers.{company.lower()}.com/jobs/{job_id}",  # Simulated direct application link
            "search_query": f"{role} in {city}"
        }
        results.append(job_entry)
    
    # Add a small delay to simulate real API calls
    time.sleep(0.1)
    
    return results

# Limit to 20 queries for demonstration (you can adjust this)
queries = queries[:20]

# Run all job searches with enhanced link functionality
all_results = []
for q in queries:
    # Use the enhanced search function
    results = search_jobs_with_api_simulation(q)  # or search_jobs_with_links(q) for single results
    all_results.extend(results)

print(f"\nJob search completed. Found {len(all_results)} results.")

# Enhanced CSV output with links
def save_results_to_csv(results, filename='enhanced_job_search_results.csv'):
    """Save job results to CSV with all link information"""
    if not results:
        print("No results to save")
        return
    
    # Get all fieldnames from the first result
    fieldnames = list(results[0].keys())
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
    
    print(f"All job search results have been saved to {filename}")

# Save to CSV
save_results_to_csv(all_results)

# Display sample results with links
print("\nSample results with links:")
print("-" * 80)
for i, result in enumerate(all_results[:5]):  # Show first 5 results
    print(f"{i+1}. {result['title']} in {result['location']}")
    print(f"   Company: {result.get('company', 'Various')}")
    if 'application_link' in result:
        print(f"   Direct Application: {result['application_link']}")
    print(f"   LinkedIn: {result['linkedin_link']}")
    print(f"   Indeed: {result['indeed_link']}")
    print(f"   Naukri: {result['naukri_link']}")
    print(f"   Glassdoor: {result['glassdoor_link']}")
    print()

# Optional: Create an HTML file with clickable links
def create_html_output(results, filename='job_search_results.html'):
    """Create an HTML file with clickable job links"""
    html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Job Search Results</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .job-card { border: 1px solid #ddd; margin: 10px 0; padding: 15px; border-radius: 5px; }
        .job-title { font-size: 18px; font-weight: bold; color: #333; }
        .job-location { color: #666; margin: 5px 0; }
        .job-links { margin: 10px 0; }
        .job-links a { margin-right: 15px; padding: 5px 10px; background: #007bff; color: white; text-decoration: none; border-radius: 3px; }
        .job-links a:hover { background: #0056b3; }
    </style>
</head>
<body>
    <h1>Job Search Results</h1>
"""
    
    for result in results:
        html_content += f"""
    <div class="job-card">
        <div class="job-title">{result['title']}</div>
        <div class="job-location">📍 {result['location']}</div>
        <div class="job-links">
            <a href="{result['linkedin_link']}" target="_blank">LinkedIn</a>
            <a href="{result['indeed_link']}" target="_blank">Indeed</a>
            <a href="{result['naukri_link']}" target="_blank">Naukri</a>
            <a href="{result['glassdoor_link']}" target="_blank">Glassdoor</a>
"""
        if 'application_link' in result:
            html_content += f'            <a href="{result[\'application_link\']}" target="_blank">Apply Direct</a>'
        
        html_content += """
        </div>
    </div>
"""
    
    html_content += """
</body>
</html>
"""
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML output created: {filename}")

# Create HTML output
create_html_output(all_results)

print(f"\nTotal jobs found: {len(all_results)}")
print("Files created:")
print("- enhanced_job_search_results.csv (CSV with all data)")
print("- job_search_results.html (HTML with clickable links)")

SyntaxError: unexpected character after line continuation character (1078553408.py, line 201)